In [139]:
import tensorflow as tf
import os
import numpy as np
from model import AlexNet

In [5]:
data_dir = 'asl_alphabet_train'
batch_size = 10
img_height = 227
img_width = 227

In [6]:
list_ds = tf.data.Dataset.list_files(str(data_dir + '/*/*.jpg'), shuffle=False)
image_count = len(list_ds)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=True)

class_names = np.array(sorted([item.name for item in os.scandir(data_dir)]))
num_classes = len(class_names)

2024-02-22 00:43:22.869836: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-22 00:43:22.872246: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
print(f"The total number of images in the dataset is {image_count}.")
print(f"The number of different classes is {num_classes}.")

The total number of images in the dataset is 87000.
The number of different classes is 29.


In [140]:
# # Splitting into training and validation sets
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

In [141]:
def preprocess_image(file_path):
    # Load the image
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=1)  # Use decode_jpeg for JPEG images
    img = tf.image.resize(img, [img_height, img_width])
    
    # Data augmentation
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.1)  # Adjust the max delta for brightness

    # Rescale pixel values
    img /= 255.0
    return img

In [142]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == class_names

In [143]:
# Map the preprocessing function and batch the data
train_ds = train_ds.map(lambda x: (preprocess_image(x), get_label(x)), num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(lambda x: (preprocess_image(x), get_label(x)), num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [154]:
input_shape = (227, 227, 1)
num_classes = 29 
model = AlexNet(input_shape=input_shape, num_classes=num_classes)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1
)

Epoch 1/10
 429/1088 [==========>...................] - ETA: 28:27 - loss: 83.1559 - accuracy: 0.0348

KeyboardInterrupt: 